In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
from jinja2 import Environment, FileSystemLoader
import pdfkit
import matplotlib.pyplot as plt
import seaborn as sns
import base64

def generate_faulty_panel_summary(crawl_data_folder, start_date, end_date, threshold):
    faulty_panels = {}
    current_date = datetime.now().date()

    # Start from 2024-08-26
    analysis_start_date = pd.to_datetime('2024-08-26')

    # Iterate through each week
    for week in pd.date_range(start=analysis_start_date, end=current_date, freq='W-Mon'):
        week_folder = os.path.join(crawl_data_folder, week.strftime('%Y-%m-%d'))

        if os.path.exists(week_folder):
            # Iterate through all files of that week
            for file_name in os.listdir(week_folder):
                if file_name.endswith('.csv'):
                    file_path = os.path.join(week_folder, file_name)
                    df = pd.read_csv(file_path, parse_dates=[0], index_col=[0])

                    # Calculate daily energy generation
                    daily_energy = df.sum() / 4 / 1000  # Assume each module's generation is in kWh

                    # Calculate average and standard deviation
                    avg_energy = daily_energy.mean()
                    std_dev = daily_energy.std()

                    # Iterate through each panel
                    for module in df.columns:
                        panel_daily_energy = daily_energy[module]
                        if panel_daily_energy < avg_energy - threshold * std_dev:
                            if module not in faulty_panels:
                                faulty_panels[module] = {
                                    'first_detected_date': week,
                                    'resolved_status': 'No',
                                    'kWh_loss': 0
                                }
                            # Calculate kWh loss
                            faulty_panels[module]['kWh_loss'] += (avg_energy - panel_daily_energy)

    # Update Resolved Status
    for module in faulty_panels.keys():
        last_week_folder = os.path.join(crawl_data_folder, current_date.strftime('%Y-%m-%d'))
        if os.path.exists(last_week_folder):
            last_week_file_path = os.path.join(last_week_folder, f'{module}.csv')
            if os.path.exists(last_week_file_path):
                df = pd.read_csv(last_week_file_path, parse_dates=[0], index_col=[0])
                last_week_daily_energy = df.sum() / 4 / 1000
                last_week_avg_energy = last_week_daily_energy.mean()
                last_week_std_dev = last_week_daily_energy.std()

                if last_week_daily_energy[module] >= last_week_avg_energy - threshold * last_week_std_dev:
                    faulty_panels[module]['resolved_status'] = 'Yes'

    return faulty_panels

def generate_status_table(low_energy_modules, start_date, end_date):
    date_range = pd.date_range(start='2024-08-26', end=end_date, freq='W-Mon')
    status_table = {'weeks': []}

    # Add week names
    for week in date_range:
        status_table['weeks'].append(f"Week {len(status_table['weeks']) + 1} ({week.strftime('%Y-%m-%d')} to {(week + pd.DateOffset(days=6)).strftime('%Y-%m-%d')})")

    all_panels = set()

    for module in low_energy_modules:
        statuses = []
        fault_count = 0  # Count of faults

        for week in date_range:
            week_folder = os.path.join(crawl_data_folder, week.strftime('%Y-%m-%d'))
            file_path = os.path.join(week_folder, f'{module}.csv')

            if os.path.exists(file_path):
                df = pd.read_csv(file_path, parse_dates=[0], index_col=[0])
                daily_energy = df[module].sum() / 4 / 1000
                if daily_energy < 1:
                    statuses.append('F')
                    fault_count += 1  # Increment fault count
                    all_panels.add(module)
                else:
                    statuses.append('N')
            else:
                statuses.append('F')
                fault_count += 1  # Increment fault count
                all_panels.add(module)

        # Handle Lasting Weeks
        if statuses[-1] == 'F':
            consecutive_failures = statuses.count('F')
            statuses.append(consecutive_failures)
        else:
            statuses.append('resolved')

        # Reorganize the status table
        status_table[module] = statuses[-3:] + [fault_count]  # Keep only the last three statuses and fault count

    return {
        'weeks': ['Last Week', 'This Week', 'Total Lasting Weeks', 'Number of weeks detected fault during the past quarter'],  # Modify column names
        **{panel: status_table.get(panel, ['N'] * 3 + [0]) for panel in all_panels}  # Fill content
    }

# Set folder path
crawl_data_folder = r'xxxx' #

# Calculate daily energy generation
def calculate_daily_energy(file_path):
    df = pd.read_csv(file_path, parse_dates=[0], index_col=[0])
    daily_energy = df.sum() / 4 / 1000
    daily_energy_df = pd.DataFrame({'Module': df.columns, 'PV generation (kWh)': daily_energy})
    return daily_energy_df.reset_index(drop=True)

def sum_daily_energy(target_station, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    date_folders = []
    for folder_name in os.listdir(crawl_data_folder):
        if os.path.isdir(os.path.join(crawl_data_folder, folder_name)):
            try:
                folder_date = pd.to_datetime(folder_name)
                if start_date <= folder_date <= end_date:
                    date_folders.append(folder_name)
            except ValueError:
                pass

    dfs = []
    for date_folder in date_folders:
        folder_path = os.path.join(crawl_data_folder, date_folder)
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.csv') and target_station in file_name:
                file_path = os.path.join(folder_path, file_name)
                daily_energy_df = calculate_daily_energy(file_path)
                dfs.append(daily_energy_df)

    result_df = pd.concat(dfs).groupby('Module').sum().reset_index()
    return result_df

def find_low_energy_pv(target_station, start_date, end_date, threshold):
    result_df2 = sum_daily_energy(target_station, start_date, end_date)
    mean_energy = result_df2['PV generation (kWh)'].mean()
    std_energy = result_df2['PV generation (kWh)'].std()

    low_energy_pv = result_df2[result_df2['PV generation (kWh)'] < (mean_energy - threshold * std_energy)]

    low_energy_modules = []
    for index, row in low_energy_pv.iterrows():
        percentage_below_average = ((row['PV generation (kWh)'] - mean_energy) / mean_energy) * 100
        low_energy_modules.append((row['Module'], round(percentage_below_average, 2)))

    return low_energy_modules

def get_pv_panel_module(station_name):
    site_info = pd.read_excel(r'C:\Users\Oscar\Desktop\API solaredge.xlsx')
    module_info = site_info.loc[site_info['Site Name'] == station_name, 'PV Panel Module'].values
    return module_info[0] if len(module_info) > 0 else None

def generate_heatmap(low_energy_data, start_date, end_date):
    heatmap_data = []

    # Generate date range
    date_range = pd.date_range(start=start_date, end=end_date)

    for station, details in low_energy_data.items():
        low_energy_modules = details['low_energy_modules']
        station_data = []

        for date in date_range:
            daily_percentages = []

            # Iterate through each module and calculate the generation for that date
            for module, _ in low_energy_modules:
                daily_energy = get_daily_energy(station, module, date)
                avg_energy = get_average_energy(station, date)
                if avg_energy > 0:
                    percentage_below_average = ((daily_energy - avg_energy) / avg_energy) * 100
                    daily_percentages.append(percentage_below_average)

            # Calculate the average percentage for that date
            if daily_percentages:
                station_data.append(sum(daily_percentages) / len(daily_percentages))
            else:
                station_data.append(0)

        heatmap_data.append(station_data)

    # Check the shape of heatmap_data
    if not heatmap_data or not all(len(row) == len(date_range) for row in heatmap_data):
        print("Error: heatmap_data is not properly filled.")
        return

    # Create heatmap
    plt.figure(figsize=(7, 8))
    sns.heatmap(heatmap_data, annot=True, cmap='Blues_r', 
                xticklabels=date_range.strftime('%m-%d'), 
                yticklabels=[station for station in low_energy_data.keys()],  # Only display Station names
                cbar_kws={"orientation": "vertical", "label": "Percentage(%) Below Average"},  # Add color bar label
                vmin=min(min(row) for row in heatmap_data),  # Set the minimum value of the color bar
                vmax=max(max(row) for row in heatmap_data))  # Set the maximum value of the color bar
    plt.title('Mean Percentage Below Average (%) of Low Generation Modules')
    plt.xlabel('Date')
    # plt.ylabel('PV Station')
    # Reduce font size
    plt.xticks(rotation=45, fontsize=8)  # Rotate x-axis labels by 45 degrees, set font size to 10
    plt.yticks(rotation=0, fontsize=8)    # Set font size for y-axis labels to 10
    plt.tight_layout()
    plt.savefig(r'C:\Users\Oscar\Downloads\lowenergyheatmap.png')  # Save heatmap
    plt.close()

    # Generate Base64 encoding of the overall heatmap (if needed)
    overall_heatmap_path = r'C:\Users\Oscar\Downloads\lowenergyheatmap.png'
    with open(overall_heatmap_path, "rb") as img_file:
        overall_heatmap_base64 = base64.b64encode(img_file.read()).decode('utf-8')

    return overall_heatmap_base64  # Return the Base64 of the overall heatmap

def generate_daily_heatmap_for_stations(low_energy_data, start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)

    for station, details in low_energy_data.items():
        low_energy_modules = details['low_energy_modules']

        heatmap_data = []
        for module, _ in low_energy_modules:
            daily_energy = []
            for date in date_range:
                daily_energy_value = get_daily_energy(station, module, date)
                daily_energy.append(daily_energy_value)

            heatmap_data.append(daily_energy)

        # Calculate daily averages
        if heatmap_data:
            average_daily_energy = [get_average_energy(station, date) for date in date_range]
            heatmap_data.append(average_daily_energy)

        # Heatmap
        num_rows = len(heatmap_data)  # Number of rows
        fig_height = max(num_rows * 0.5, 2)  # Each row is 0.5 inches, minimum height is 2 inches
        plt.figure(figsize=(7, fig_height))  # Fixed width of 7
        plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.2)

        # Custom formatting function
        def format_func(x):
            if x.is_integer():
                return f"{int(x)}"
            else:
                return f"{x:.1f}"  # Keep one decimal place

        # Use `annot` parameter and custom formatting function
        sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='YlOrRd', 
                    xticklabels=date_range.strftime('%m-%d'), 
                    yticklabels=[module for module, _ in low_energy_modules] + ['Average'],
                    cbar_kws={"orientation": "vertical", "label": "Energy (kWh)", "shrink": 0.5},
                    annot_kws={"size": 8})  # Set the font size for annotation text

        # Set annotation text
        for i in range(len(heatmap_data)):
            for j in range(len(heatmap_data[i])):
                value = heatmap_data[i][j]
                if isinstance(value, float) and value.is_integer():  
                    text = f"{int(value)}"
                else:
                    text = f"{value:.2f}"  # Keep one decimal place
                plt.text(j + 0.5, i + 0.5, text, ha='center', va='center', fontsize=8, color='black')  

        plt.title(f'Daily Energy of Low Generation Modules for {station}', fontsize=12)  # Uniform title font size
        plt.xlabel('Date', fontsize=10)  # Uniform x-axis label font size
        # plt.ylabel('PV Module', fontsize=10)  # Uniform y-axis label font size
        plt.xticks(rotation=45, fontsize=8)  # x-axis label font size
        plt.yticks(rotation=0, fontsize=8)  # y-axis label font size
        plt.tight_layout()        

        # Save heatmap image
        heatmap_path = os.path.join(r'C:\Users\Oscar\Desktop\Documents\hkust\PV dead detection\images', f'{station}_daily_heatmap.png')
        plt.savefig(heatmap_path)
        plt.close()

def get_daily_energy(station, module, date):
    folder_path = os.path.join(crawl_data_folder, date.strftime('%Y-%m-%d'))
    file_path = os.path.join(folder_path, f'{station}.csv')

    if os.path.exists(file_path):
        df = pd.read_csv(file_path, parse_dates=[0], index_col=[0])
        if module in df.columns:
            return df[module].sum() / 4 / 1000  # Calculate the daily energy generation (kWh)
    return 0  # Return 0 if the file does not exist or the module is not in the data

def get_average_energy(station, date):
    folder_path = os.path.join(crawl_data_folder, date.strftime('%Y-%m-%d'))
    file_path = os.path.join(folder_path, f'{station}.csv')

    if os.path.exists(file_path):
        df = pd.read_csv(file_path, parse_dates=[0], index_col=[0])
        daily_energy = df.sum() / 4 / 1000  # Calculate the energy generation of all modules (kWh)
        return daily_energy.mean()  # Return the average value
    return 0  # Return 0 if the file does not exist

def main():
    # Read total panel count
    total_panels_df = pd.read_excel(r'C:\Users\Oscar\Desktop\API solaredge.xlsx', usecols=['Site Name', 'Number'])
    total_panels_dict = dict(zip(total_panels_df['Site Name'], total_panels_df['Number']))
    today = datetime.now()
    start_date = (today - timedelta(days=today.weekday())).date()  # This week's Monday
    end_date = today.date()  # This week's Sunday

    site_info = pd.read_excel(r'C:\Users\Oscar\Desktop\API solaredge.xlsx')
    site_list = site_info['Site Name']

    low_energy_data = {}
    threshold = 2  # Adjust the threshold as needed

    for target_station in site_list:
        low_energy_modules = find_low_energy_pv(target_station, start_date, end_date, threshold)
        if low_energy_modules:  # If there are low-energy panels
            pv_module = get_pv_panel_module(target_station)
            low_energy_data[target_station] = {
                'module': pv_module,
                'low_energy_modules': sorted(low_energy_modules, key=lambda x: x[1])  # Sort by severity
            }

    generate_heatmap(low_energy_data, start_date, end_date)  # Generate weekly heatmap
    generate_daily_heatmap_for_stations(low_energy_data, start_date, end_date)  # Generate daily heatmap
    generate_report(low_energy_data, start_date, end_date, total_panels_dict, threshold)  # Generate report

def generate_report(low_energy_data, start_date, end_date, total_panels_dict, threshold):
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template('report_template.html')

    # Generate Base64 encoding of the overall heatmap

def generate_report(low_energy_data, start_date, end_date, total_panels_dict, threshold):
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template('report_template.html')

    # Generate Base64 encoding of the overall heatmap
    overall_heatmap_base64 = generate_heatmap(low_energy_data, start_date, end_date)

    # Generate Faulty Panel Summary
    faulty_panel_summary = generate_faulty_panel_summary(crawl_data_folder, start_date, end_date, threshold)

    # Collect kWh_loss in a list
    kWh_losses = [details['kWh_loss'] for details in faulty_panel_summary.values()]
    total_kWh_loss = sum(kWh_losses)  # Calculate the total

    # Print for checking
    print(f"Total kWh Loss: {total_kWh_loss}")

    # Count the number of PV panels for each model
    module_count = {}
    station_summary = {}

    for station, details in low_energy_data.items():
        low_energy_count = len(details['low_energy_modules'])
        total_panels = total_panels_dict.get(station, 0)

        low_energy_percentage = (low_energy_count / total_panels) * 100 if total_panels > 0 else 0

        station_summary[station] = {
            'low_energy_count': low_energy_count,
            'total_panels': total_panels,
            'low_energy_percentage': low_energy_percentage
        }

        module_name = details['module']
        module_count[module_name] = module_count.get(module_name, 0) + low_energy_count

        details['heatmap_path'] = os.path.abspath(os.path.join(r'C:\Users\Oscar\Desktop\Documents\hkust\PV dead detection\images', f'{station}_daily_heatmap.png'))

        with open(details['heatmap_path'], "rb") as img_file:
            details['daily_heatmap_base64'] = base64.b64encode(img_file.read()).decode('utf-8')

        # Generate status table
        details['status_table'] = generate_status_table(details['low_energy_modules'], start_date, end_date)

    # Render HTML content
    html_content = template.render(
        low_energy_data=low_energy_data,
        start_date=start_date,
        end_date=end_date,
        station_summary=station_summary,
        module_count=module_count,
        overall_heatmap_base64=overall_heatmap_base64,
        faulty_panel_summary=faulty_panel_summary,
        total_kWh_loss=total_kWh_loss  # New content
    )

    with open('weekly_report.html', 'w', encoding='utf-8') as f:
        f.write(html_content)

    # Ensure the path to wkhtmltopdf is correct
    path_to_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'
    config = pdfkit.configuration(wkhtmltopdf=path_to_wkhtmltopdf)

    options = {
        'disable-smart-shrinking': None,
        'no-stop-slow-scripts': None
    }

    # Generate PDF
    pdfkit.from_file('weekly_report.html', 'weekly_report.pdf', configuration=config, options=options)

if __name__ == "__main__":
    main()
    
